In [1]:
# Import modules
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time

In [2]:
# URL of page to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'

In [4]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
browser.visit(url)

In [6]:
tables = pd.read_html(url)

In [7]:
tables

[    Rank   Peak                                          Title  \
 0      1      1                              Avengers: Endgame   
 1      2      1                                         Avatar   
 2      3      1                                        Titanic   
 3      4      3                   Star Wars: The Force Awakens   
 4      5      4                         Avengers: Infinity War   
 5      6      3                                 Jurassic World   
 6      7      7                                  The Lion King   
 7      8      3                                   The Avengers   
 8      9      4                                      Furious 7   
 9     10     10                                      Frozen II   
 10    11      5                        Avengers: Age of Ultron   
 11    12      9                                  Black Panther   
 12    13      3  Harry Potter and the Deathly Hallows – Part 2   
 13    14      9                       Star Wars: The Last Jed

In [8]:
table_df = tables[0]

In [9]:
table_df

,Rank,Peak,Title,Worldwide gross,Year,Reference(s)
0,1,1,Avengers: Endgame,"$2,797,800,564",2019,[# 1][# 2]
1,2,1,Avatar,"$2,790,439,000",2009,[# 3][# 4]
2,3,1,Titanic,"$2,194,439,542",1997,[# 5][# 6]
3,4,3,Star Wars: The Force Awakens,"$2,068,223,624",2015,[# 7][# 8]
4,5,4,Avengers: Infinity War,"$2,048,359,754",2018,[# 9][# 10]
5,6,3,Jurassic World,"$1,671,713,208",2015,[# 11][# 12]
6,7,7,The Lion King,"$1,656,943,394",2019,[# 13][# 2]
7,8,3,The Avengers,"$1,518,812,988",2012,[# 14][# 15]
8,9,4,Furious 7,"$1,516,045,911",2015,[# 16][# 17]
9,10,10,Frozen II,"$1,450,026,933",2019,[# 18]


In [ ]:
table_df.to_csv()